# Generate data using api

In [25]:
df = pd.read_csv(r".\data\released_data_mentalllama\train_data\complete_data\DR\reddit_valid.csv")

df.question.unique()

array([' Question: The answer to the question "Does the poster suffers from depression?" is '],
      dtype=object)

#### Sampling without replacement from original data

In [26]:
import random
from update_prompt import update_prompt_with_csv


prompt_path = r".\prompts\prompt_complete_DR.txt"
csv_valid_path = r".\data\released_data_mentalllama\train_data\complete_data\DR\reddit_valid.csv"
csv_train_path = r".\data\released_data_mentalllama\train_data\complete_data\DR\reddit_train.csv"

random.seed(42) # For reproducibility
n = 25


update_prompt_with_csv(prompt_path, csv_valid_path, csv_train_path, n)

Updated '.\prompts\prompt_complete_DR.txt' with 25 new example rows.


#### data generation

In [8]:
from openai import OpenAI

client_ds = OpenAI(api_key="sk-ed8f7a41078845a5bf6d6db82686ca08", base_url="https://api.deepseek.com")

In [27]:
with open('./prompts/prompt_complete_DR.txt', 'r') as file:
    content = file.read()


response = client_ds.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a mental health data generation assistant"},
        {"role": "user", "content": content},
    ],
    stream=False,
    temperature=1.6,
    max_tokens=500
)

print(response.choices[0].message.content)

"Post: I feel like I’m drowning in my own thoughts. Every day is a battle to get out of bed, and when I finally do, I’m hit with this overwhelming sense of emptiness. I try to distract myself with work, but it feels like I’m just going through the motions. I can’t remember the last time I genuinely laughed or felt happy. My friends keep telling me to ‘just cheer up,’ but they don’t understand how hard it is to even pretend to be okay. I feel so alone, like no one really gets it. I’ve started avoiding social situations because I don’t want to bring anyone down with my negativity. I don’t know how much longer I can keep this up.","Question: The answer to the question 'Does the poster suffer from depression?' is","yes. Reasoning: The post exhibits strong emotional indicators of depression. The poster describes a persistent sense of emptiness, difficulty getting out of bed, and a lack of enjoyment in activities they once found pleasurable. They also express feelings of isolation and a beli

In [30]:
import csv
import io
import os

reader = csv.reader(io.StringIO(response.choices[0].message.content))
rows = list(reader)

header = ['post', 'question', 'response']
output_csv = './data/data_generated/DR_generated.csv'

file_exists = os.path.exists(output_csv)

with open(output_csv, 'a' if file_exists else 'w', newline='', encoding='utf-8') as out_f:
    writer = csv.writer(out_f)
    if not file_exists or os.stat(output_csv).st_size == 0:
        writer.writerow(header)
    writer.writerows(rows)

print(f"CSV file '{output_csv}' updated with {len(rows)} new rows of data.")

CSV file './data/data_generated/DR_generated.csv' updated with 3 new rows of data.


In [ ]:
# data prepossessing
pd.read_csv('./data/data_generated/DR_generated.csv')

,post,question,response
0,Post: I feel like I’m drowning in my own thoug...,Question: The answer to the question 'Does the...,yes. Reasoning: The post exhibits strong emoti...
1,Post: I’ve been feeling so off lately. I can’t...,Question: The answer to the question 'Does the...,no. Reasoning: While the post describes signif...
